#Ecuación Unidimensional de Burguers'

$$
\frac{\partial \psi}{\partial t} + u\frac{\partial \psi}{\partial x} = \nu \frac{\partial^2\psi}{\partial x^2}
$$

En esta ecuación tenemos la combinación de convección y difusión y esta describiendo como atravez del tiempo la propiedad del fluido $\psi$ se **convecta** y se **difunde** en el espacio unidimensional. Para nuestro caso nos interesa la convección y difusión de la **velocidad**.

$$
\frac{\partial u}{\partial t} + u\frac{\partial u}{\partial x} = \nu \frac{\partial^2u}{\partial x^2}
$$

Ahora la discretización para la ecuación de Burgers' es 

$$
\frac{u(i)_{n+1} - u(i)_n}{\Delta t} + u(i)_n \frac{u(i)_n - u(i-1)_n}{\Delta x} = \nu \frac{u(i+1)_n - 2u(i)_n + u(i-1)_n}{\Delta x^2}
$$

De nuevo resolvemos para la variable desconocida, $u(i)_{n+1}$

$$
u(i)_{n+1} = u(i)_n + \Delta t \Bigg[ \nu \frac{u(i+1)_n - 2u(i)_n + u(i-1)_n}{\Delta x^2} - u(i)_n \frac{u(i)_n - u(i-1)_n}{\Delta x} \Bigg]
$$

$$
u(i)_{n+1} = u(i)_n - \frac{u(i)_n \Delta t}{\Delta x} \Bigg[ u(i)_n - u(i-1)_n \Bigg]  + \frac{\nu \Delta t}{\Delta x^2}  \Big[u(i+1)_n - 2u(i)_n + u(i-1)_n \Bigg]
$$

Para este problema impondremos como condición inicial utilizando la transformada de **Hopf-Cole**:

$$
u(x) = -\frac{2 \nu}{\phi} \frac{\partial \phi}{\partial x} + 4
$$

Donde:

$$
\phi(x) = \exp \bigg(\frac{-x^2}{4 \nu} \bigg) + \exp \bigg(\frac{-(x-2 \pi)^2}{4 \nu} \bigg)
$$

Para los valores en las fronteras:

$$
u(0) = u(2\pi)
$$

Para este problema vamos a comparar gráficamente la solución analítica con la numérica.

In [1]:
import numpy as np;
import sympy as smp;
from IPython.display import display, Math, Latex

In [2]:
x, t, nu = smp.symbols('x t nu');

# define the psi function in sympy notation
psi = smp.exp(-(x-4*t)**2 / (4*nu*(t+1))) + smp.exp( -((x-4*t) - 2*smp.pi)**2 / (4*nu*(t+1)) );

# take the partial derivative respect to x
psi_partial_x = smp.diff(psi, x);

###Versión sympy para $\psi(x,t)$

In [3]:
display( Math(smp.latex(psi)) );

<IPython.core.display.Math object>

###Versión sympy para $\frac{\partial \psi}{\partial x}$

In [4]:
display( Math(smp.latex(psi_partial_x)) );

<IPython.core.display.Math object>

In [5]:
# this function converts a sympy function into a numpy function
def lamb(function):
    return smp.lambdify((x, t, nu), function, modules='numpy');

# from here just do the conversion
psi, psi_partial_x = map(lamb, (psi, psi_partial_x));

# Frome here the functions psi & psi_partial_x are callable as we are used

In [8]:
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np;

# How much we want to advance in time?
# at 30 fps if we want simulate 10 seconds => 30*10
t_seconds = 30 * 60;

# The time step size 
dt = 0.005;

# spatial partitions
x_partitions = 100;

data = np.linspace(0.0, 2.0*np.pi, num=x_partitions, retstep=True);

x  = data[0];
dx = data[1];

# Set initial profile
nu = 0.07;
time = 0;
u = -((2.0 * nu) / psi(x, time, nu)) * psi_partial_x(x, time, nu) + 4;
u_analytical = -((2.0 * nu) / psi(x, time, nu)) * psi_partial_x(x, time, nu) + 4;

fig = plt.figure();
fig.suptitle(r'$\frac{\partial u}{\partial t} + u\frac{\partial u}{\partial x} = \nu \frac{\partial^2 u}{\partial x^2}$', fontsize=20, fontweight='bold');
ax = plt.axes(xlim=(0.0, 2.0*np.pi), ylim=(-0.25, 10.0));
profile, = ax.plot([], [], lw=2);
analytical_profile, = ax.plot([], [], lw=1, color='r');

def init():
    profile.set_data([], []);
    analytical_profile.set_data([], []);
    
    return profile, analytical_profile

def update(n): 
    
    if(n == 0):
        profile.set_data(x, u);
        analytical_profile.set_data(x, u_analytical);
        return profile,        
    
    #store the actual state of the array
    u_n = u;
    
    #update array
    u[1:] = u_n[1:] - ((u_n[1:]*dt)/dx) * ( u_n[1:] - np.roll(u_n, 1)[1:] ) + ((nu * dt)/dx**2) * (np.roll(u_n, -1)[1:] - 2*u_n[1:] + np.roll(u_n, 1)[1:]);
    u[0] = u[-1];
    profile.set_data(x, u);
    
    time = n*dt;
    psi_n = -((2.0 * nu) / psi(x, time, nu)) * psi_partial_x(x, time, nu) + 4;
    analytical_profile.set_data(x, psi_n);

    return profile, u_analytical

# The following lines works only on a machine with ffmpeg configured
anim = animation.FuncAnimation(fig, update, init_func=init, frames=t_seconds, blit=True);
anim.save('1D_Burguers_Equation_01.mp4', fps=30, writer="ffmpeg", codec="libx264");
# plt.show(fig);
plt.close(fig);

C:\dev-libs\Anaconda\lib\site-packages\IPython\kernel\__main__.py:54: RuntimeWarning: overflow encountered in divide


In [9]:
# You must have configured ffmpeg in your machine to run the code below
from IPython.display import HTML
video = open("1D_Burguers_Equation_01.mp4", "rb").read()
video_encoded = video.encode("base64")
video_tag = '<video controls alt="test" src="data:video/x-m4v;base64,{0}">'.format(video_encoded)
HTML(video_tag)

Para la solución numérica, vemos un efecto de repetición una vez el profile ha salido de la región, esto se debe a las condiciones de frontera que hemos fijado $u(0) = u(2\pi)$, las cuales implican que para las iteraciones en las frontera por la izquierda los nuevos calculos se van a hacer con respecto de la frontera de salida por la derecha. 